# 6 - Методи обчислень
## Лабораторна робота 2
> Михайло Корешков, ФІ-91

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
LOG = True
D = 6

def log(*args, a=None, **kwargs):
    if LOG and (a is None or a):
        print(*args, **kwargs)

# Зворотній хід

In [3]:
def backstep_U(U, b):
    n = U.shape[1]
    x = np.zeros(n)
    x[n-1] = round(b[n-1] / U[n-1,n-1], D)
    for i in range(n-2, -1, -1):
        x[i] = ((b[i] - U[i,i+1:] @ x[i+1:]) / U[i,i]).round(D)
    return x

def backstep_L(L, b):
    n = L.shape[1]
    x = np.zeros(n)
    x[0] = round(b[0] / L[0,0], D)
    for i in range(1,n):
        x[i] = ((b[i] - L[i,:i] @ x[:i]) / L[i,i]).round(D)
    return x

In [4]:
LOG = True
U = np.triu(np.random.rand(4,4))
x0 = np.random.rand(4)
b = U @ x0
x = backstep_U(U,b)

print(x - x0)

[ 5.01157922e-05 -1.08661339e-05  2.79179162e-07  4.69562815e-07]


In [5]:
LOG = True
L = np.tril(np.random.rand(4,4))
x0 = np.random.rand(4)
b = L @ x0
x = backstep_L(L,b)

print(x - x0)

[ 4.58762347e-07 -4.79659628e-07  7.21006510e-07 -7.74765917e-06]


# Метод 1: "Квадратного кореня"
Використовується для симетричних матриць

In [6]:
# 1. Факторизація симетричної додатньо визначеної A = LL'

def LU_sym_factor(_A):
    A = _A.astype(float)
    n = A.shape[0]
    L = np.zeros_like(A).astype(float)
    for i in range(0, n):
        for j in range(0,i):
            L[i,j] = ((A[i,j] - L[i,0:i] @ L[j,0:i]) / L[j,j]).round(D)
        L[i,i] = np.sqrt(A[i,i] - L[i,0:i] @ L[i,0:i]).round(D)
        log(L)
    return L, L.T

In [7]:
# 2. Зворотній хід. Розв'язує систему LL'x = n; (Ly=b, L'x=y)
def solve_LL(L, b):
    n = L.shape[0]
    
    y = backstep_L(L, b)
    log("y:", y)
    
    x = backstep_U(L.T, y)
    
    return x

# Метод 2: Гауса з півотами

In [8]:
def solve_gauss(_A, _b):
    A = _A.copy().astype(float)
    b = _b.copy().astype(float)
    
    n = A.shape[0]
    # permutations
    pi = np.arange(n)
    pj = np.arange(n)
    # offset
    ki, kj = 0, 0
    steps = 1
    while ki < n-1 and kj < n-1:
        # find the largest element of the Bottom Right (BR)
        M, absM = 0, 0
        Mi, Mj = ki, kj
        for i in range(ki,n):
            for j in range(kj,n):
                a = np.abs(A[i,j])
                if a > M: 
                    absM, M, Mi, Mj = a, A[i,j], i, j
        
        # save permutation
        pi[ki], pi[Mi] = pi[Mi], pi[ki]
        pj[kj], pj[Mj] = pj[Mj], pj[kj]
        
        # swap rows and b
        r1, r2 = A[ki], A[Mi]
        A[ki], A[Mi] = r2.copy(), r1.copy()
        
        b1,b2 = b[ki], b[Mi]
        b[ki], b[Mi] = b2,b1
        
        # swap cols and x
        c1, c2 = A[:,kj], A[:,Mj]
        A[:,kj], A[:,Mj] = c2.copy(), c1.copy()
        
        log("swap rows and cols")
        log(np.hstack([A, b.reshape(-1,1)]))
        
        BR = A[ki:,kj:]
        
        b[ki] = round(b[ki] / M, D)
        BR[0] = (BR[0] / M).round(D)
        
        log("normalize")
        log(np.hstack([A, b.reshape(-1,1)]))

        
        b[ki+1:] -= b[ki] * BR[1:,0]
        BR[1:] -= BR[0] * BR[1:,0].reshape(-1,1)

        log("subtract")
        log(np.hstack([A, b.reshape(-1,1)]))

        
        ki += 1
        kj += 1
        steps += 1
    
    # last step
    b[-1] = round(b[-1] / A[-1,-1], D)
    A[-1,-1] = 1

    log("last step - normalize")
    log(np.hstack([A, b.reshape(-1,1)]))

    # зворотній хід
    x = backstep_U(A, b)
    
    # inverse permutation
    ipj = sorted(list(zip(pj, range(n))), key=lambda tup: tup[0])
    ipj = [t[1] for t in ipj]
    x = x[ipj]
    
    return x
        

In [9]:
LOG = False

A = np.array([[1,2,3],[4,5,6],[7,8,10]])
x0 = np.random.rand(3)
# b = np.array([120,120,120])
b = A @ x0

print(np.hstack([A,b.reshape(-1,1)]))
x = solve_gauss(A, b)

print(x)
print(x0)
print(x - x0)

[[ 1.          2.          3.          4.39458943]
 [ 4.          5.          6.         10.70141583]
 [ 7.          8.         10.         17.71421031]]
[0.51593  0.880379 0.705967]
[0.51592959 0.88037779 0.70596808]
[ 4.10034144e-07  1.20649990e-06 -1.08338574e-06]


# Завдання
> Варіант 6 

In [10]:
LOG = True
D = 7

A = np.array([
    [5.5, 7., 6., 5.5],
    [7., 10.5, 8., 7.,],
    [6.0, 8.0, 10.5, 9.],
    [5.5, 7., 9., 10.5]
])
b = np.array([23, 32, 33, 31])

In [11]:
# square root method
L, U = LU_sym_factor(A)
x1 = solve_LL(L, b)
r1 = A @ x1 - b
print()
print("x = ", x1)
print("Нев'язка: ", r1)

[[2.3452079 0.        0.        0.       ]
 [0.        0.        0.        0.       ]
 [0.        0.        0.        0.       ]
 [0.        0.        0.        0.       ]]
[[2.3452079 0.        0.        0.       ]
 [2.98481   1.2613125 0.        0.       ]
 [0.        0.        0.        0.       ]
 [0.        0.        0.        0.       ]]
[[2.3452079 0.        0.        0.       ]
 [2.98481   1.2613125 0.        0.       ]
 [2.5584086 0.2883    1.9675946 0.       ]
 [0.        0.        0.        0.       ]]
[[2.3452079 0.        0.        0.       ]
 [2.98481   1.2613125 0.        0.       ]
 [2.5584086 0.2883    1.9675946 0.       ]
 [2.3452079 0.        1.5247043 1.6356273]]
y: [9.8072329 2.1622502 3.7028534 1.439352 ]

x =  [0.1599997 1.4400002 1.2       0.88     ]
Нев'язка:  [-2.50000003e-07  0.00000000e+00 -2.00000002e-07 -2.49999999e-07]


In [12]:
# gauss method
x2 = solve_gauss(A, b)
r2 = A @ x2 - b
print()
print("x = ", x2)
print("Нев'язка: ", r2)

swap rows and cols
[[10.5  7.   8.   7.  32. ]
 [ 7.   5.5  6.   5.5 23. ]
 [ 8.   6.  10.5  9.  33. ]
 [ 7.   5.5  9.  10.5 31. ]]
normalize
[[ 1.         0.6666667  0.7619048  0.6666667  3.047619 ]
 [ 7.         5.5        6.         5.5       23.       ]
 [ 8.         6.        10.5        9.        33.       ]
 [ 7.         5.5        9.        10.5       31.       ]]
subtract
[[1.        0.6666667 0.7619048 0.6666667 3.047619 ]
 [0.        0.8333331 0.6666664 0.8333331 1.666667 ]
 [0.        0.6666664 4.4047616 3.6666664 8.619048 ]
 [0.        0.8333331 3.6666664 5.8333331 9.666667 ]]
swap rows and cols
[[1.        0.6666667 0.7619048 0.6666667 3.047619 ]
 [0.        5.8333331 3.6666664 0.8333331 9.666667 ]
 [0.        3.6666664 4.4047616 0.6666664 8.619048 ]
 [0.        0.8333331 0.6666664 0.8333331 1.666667 ]]
normalize
[[1.        0.6666667 0.7619048 0.6666667 3.047619 ]
 [0.        1.        0.6285714 0.1428571 1.657143 ]
 [0.        3.6666664 4.4047616 0.6666664 8.619048 ]
 [